# Task 1: Using RLTK to perform Entity Resolution (ER)

<sub>Content of this notebook was prepared by Basel Shbita, and modified by Avijit Thawani (thawani@usc.edu) as part of the class <u>DSCI 558: Building Knowledge Graphs</u> at University of Southern California (USC).</sub>

The Record Linkage ToolKit ([RLTK](https://github.com/usc-isi-i2/rltk)) is a general-purpose open-source record linkage platform that allows users to build powerful Python programs that link records referring to the same underlying entity.

This notebook introduces some applied examples using RLTK. You can also find additional examples and use-cases in [RLTK's documentation](https://rltk.readthedocs.io/en/master/).

## Dataset analysis & RLTK components construction

In [ ]:
!pip install rltk

### Task 1-1. Construct RLTK Datasets

First, you need define how a single entry would like for each type of record (for each dataset)

In [1]:
import rltk
import csv

# You can use this tokenizer in case you need to manipulate some data
tokenizer = rltk.tokenizer.crf_tokenizer.crf_tokenizer.CrfTokenizer()

In [2]:
'''
Feel free to add more columns here for use in record linkage.
'''

class GoodRecord(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = ''

    @rltk.cached_property
    def id(self):
        return self.raw_object['ID']

    @rltk.cached_property
    def name_string(self):
        return self.raw_object['Title']

    @rltk.cached_property
    def name_tokens(self):
        return set(tokenizer.tokenize(self.name_string))

class NobleRecord(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = ''

    @rltk.cached_property
    def id(self):
        return self.raw_object['ID']

    @rltk.cached_property
    def name_string(self):
        return self.raw_object['Title']
    
    @rltk.cached_property
    def name_tokens(self):
        return set(tokenizer.tokenize(self.name_string))

In [3]:
dir_ = ''
good_file = dir_ + 'goodreads.csv'
noble_file = dir_ + 'barnes_and_nobles.csv'

ds1 = rltk.Dataset(rltk.CSVReader(open(good_file, encoding='utf-8')), record_class=GoodRecord)
ds2 = rltk.Dataset(rltk.CSVReader(open(noble_file, encoding='utf-8')), record_class=NobleRecord)

You can load your csv files into RLTK using this method:

And we can inspect a few entries:

In [4]:
# print some entries
print(ds1.generate_dataframe().head(5))
print(ds2.generate_dataframe().head(5))

  id                                 name_string  \
0  0          Managing My Life: My Autobiography   
1  1     I Remember: Sketch for an Autobiography   
2  2              Betty Boothroyd: Autobiography   
3  3  Caddie, A Sydney Barmaid: An Autobiography   
4  4     Nureyev: An Autobiography With Pictures   

                                         name_tokens  
0             {Autobiography, My, Life, Managing, :}  
1   {Autobiography, for, an, Sketch, I, Remember, :}  
2               {Boothroyd, Betty, Autobiography, :}  
3  {Autobiography, ,, A, Sydney, Barmaid, :, An, ...  
4    {Autobiography, Pictures, Nureyev, With, An, :}  
  id                                        name_string  \
0  0          Pioneer Girl: The Annotated Autobiography   
1  1  American Sniper (Movie Tie-in Edition): The Au...   
2  2                     The Autobiography of Malcolm X   
3  3                           Assata: An Autobiography   
4  4                            Autobiography of a Yogi   

  

### Task 1-2. Blocking

First, we'll load dev set to evaluate both blocking (Task 1-2) and entity linking (Task 1-3).

In [24]:
print('inside blocks:')
i = 0
for b, d, r in block:
    print(b, d, r)
    i += 1
    if i == 10: break

inside blocks:
Managing My Life: My Autobiography GoodRecord 0
Managing My Life: My Autobiography NobleRecord 409
I Remember: Sketch for an Autobiography GoodRecord 1
Betty Boothroyd: Autobiography GoodRecord 2
Caddie, A Sydney Barmaid: An Autobiography GoodRecord 3
Nureyev: An Autobiography With Pictures GoodRecord 4
Soul On The Street: An Autobiography GoodRecord 5
Autobiography (Hanuman Book No. 40) GoodRecord 6
Autobiography: My Story GoodRecord 7
Jenny Pitman: The Autobiography GoodRecord 8


In [25]:
bg = rltk.HashBlockGenerator()
block = bg.generate(
    bg.block(ds1, property_='name_string'),
    bg.block(ds2, property_='name_string')
)
pairs = rltk.get_record_pairs(ds1, ds2, block=block)
i = 0
for r1, r2 in pairs:
    print(r1.id,type(r1.id), r1.name_string, '\t', r2.id, r2.name_string)
    i += 1
print(i)

0 <class 'str'> Managing My Life: My Autobiography 	 409 Managing My Life: My Autobiography
8 <class 'str'> Jenny Pitman: The Autobiography 	 1431 Jenny Pitman: The Autobiography
11 <class 'str'> Call Me Anna: The Autobiography of Patty Duke 	 100 Call Me Anna: The Autobiography of Patty Duke
13 <class 'str'> Chasing the Wind: The Autobiography of Steve Fossett 	 1152 Chasing the Wind: The Autobiography of Steve Fossett
22 <class 'str'> Mad, Bad & Dangerous to Know: The Autobiography 	 192 Mad, Bad & Dangerous to Know: The Autobiography
26 <class 'str'> William Barclay: A Spiritual Autobiography 	 1280 William Barclay: A Spiritual Autobiography
27 <class 'str'> Second Chance: The Autobiography 	 3307 Second Chance: The Autobiography
28 <class 'str'> To the Stars: The Autobiography of George Takei, Star Trek's Mr. Sulu 	 356 To the Stars: The Autobiography of George Takei, Star Trek's Mr. Sulu
33 <class 'str'> Autobiography of a Spiritually Incorrect Mystic 	 586 Autobiography of a Spir

In [21]:
len(pairs)

TypeError: object of type 'generator' has no len()

In [27]:
dev_set_file = dir_ + 'dev.csv'
dev = []
with open(dev_set_file, encoding='utf-8', errors="replace") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if len(row) <= 1:
            continue
        if line_count == 0:
            columns = row
            line_count += 1
        else:
            dev.append(row)
    print(f'Column names are: {", ".join(columns)}')
    print(f'Processed {len(dev)} lines.')

gt = rltk.GroundTruth()
for row in dev:    
    r1 = ds1.get_record(row[0])
    r2  = ds2.get_record(row[1])
    if row[-1] == '1':
        gt.add_positive(r1.raw_object['ID'], r2.raw_object['ID'])
    else:
        gt.add_negative(r1.raw_object['ID'], r2.raw_object['ID'])

rltk.Trial(gt)

Column names are: goodreads.ID, barnes_and_nobles.ID, label
Processed 297 lines.


Then, you can build your own blocking techniques and evaluate it.

Hint:

- What is the total number of pairs without blocking? 
- what is the number of paris with blocking?
- After blocking, how many "correct" (matched) pairs presented in dev set?


### Task 1-3. Entity Linking

Here are 2 example functions for field (attribute) similarity:

In [ ]:
def name_string_similarity_1(r1, r2):
    ''' Example dummy similiary function '''
    s1 = r1.name_string[:3]
    s2 = r2.name_string[:3]
    
    return rltk.jaro_winkler_similarity(s1, s2)
    
def name_string_similarity_2(r1, r2):
    ''' Example dummy similiary function '''
    s1 = r1.name_string
    s2 = r2.name_string
    
    if s1 == s2:
        return 1
    
    return 0

Here's how you can combine multiple similarity functions into a single weightened scoring function:

In [ ]:
# threshold value to determine if we are confident the record match
MY_TRESH = 0.8 # this number is just an example, you need to change it

# entity linkage scoring function
def rule_based_method(r1, r2):
    score_1 = name_string_similarity_1(r1, r2)
    score_2 = name_string_similarity_2(r1, r2)
    
    total = 0.7 * score_1 + 0.3 * score_2
    
    # return two values: boolean if they match or not, float to determine confidence
    return total > MY_TRESH, total

Lets run some candidates using the ground-truth

In [ ]:
trial = rltk.Trial(gt)
candidate_pairs = rltk.get_record_pairs(ds1, ds2, ground_truth=gt)
for r1, r2 in candidate_pairs:
    result, confidence = rule_based_method(r1, r2)
    trial.add_result(r1, r2, result, confidence)

Now lets evaluate our trial results

In [ ]:
trial.evaluate()
print('Trial statistics based on Ground-Truth from development set data:')
print(f'tp: {trial.true_positives:.06f} [{len(trial.true_positives_list)}]')
print(f'fp: {trial.false_positives:.06f} [{len(trial.false_positives_list)}]')
print(f'tn: {trial.true_negatives:.06f} [{len(trial.true_negatives_list)}]')
print(f'fn: {trial.false_negatives:.06f} [{len(trial.false_negatives_list)}]')

Trial statistics based on Ground-Truth from development set data:
tp: 0.597015 [40]
fp: 0.052174 [12]
tn: 0.947826 [218]
fn: 0.402985 [27]


In [ ]:
trial.f_measure

0.6722689075630253

### Save Test predictions
You will be evaluated on dev and test predictions, over a hidden ground truth.

In [ ]:
test_set_file = dir_ + 'test.csv'
test = []
with open(test_set_file, encoding='utf-8', errors="replace") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if len(row) <= 1:
            continue
        if line_count == 0:
            columns = row
            line_count += 1
        else:
            test.append(row)
    print(f'Column names are: {", ".join(columns)}')
    print(f'Processed {len(test)} lines.')

Column names are: goodreads.ID, barnes_and_nobles.ID
Processed 100 lines.


In [ ]:
predictions = []
for id1, id2 in test:
    r1 = ds1.get_record(id1)
    r2  = ds2.get_record(id2)
    result, confidence = rule_based_method(r1, r2)
    predictions.append((r1.id, r2.id, result, confidence))

In [ ]:
len(predictions), len(ds1.generate_dataframe()), len(ds2.generate_dataframe())

(100, 3967, 3701)

In [ ]:
with open(dir_ + 'Zhenmin_Hua_predictions.csv', mode='w') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for row in predictions:
        writer.writerow(row)

# Task 2: Using RDFLib for Knowledge Representation

RDFLib is a Python library for working with RDF, a simple yet powerful language for representing information as graphs. RDFLib aims to be a pythonic RDF API, a Graph is a python collection of RDF Subject, Predicate,  Object Triples.

This notebook introduces simple examples. You can also find additional information in the [official documenation](https://rdflib.readthedocs.io/en/stable/).

In [32]:
! pip install rdflib

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ------------------------------------- 500.3/500.3 kB 49.0 kB/s eta 0:00:00
     --------------------------------------- 41.7/41.7 kB 49.1 kB/s eta 0:00:00


In [4]:
from rdflib import Graph, URIRef, Literal, XSD, Namespace, RDF

Let's define some namespaces:

In [5]:
FOAF = Namespace('http://xmlns.com/foaf/0.1/')
MYNS = Namespace('http://dsci558.org/myfakenamespace#')

We can create a graph:

In [6]:
my_kg = Graph()
my_kg.bind('myns', MYNS)
my_kg.bind('foaf', FOAF)

Define a URI, then add a simple triple to the graph:

In [7]:
node_uri = URIRef(MYNS['dsci_558'])
my_kg.add((node_uri, RDF.type, MYNS['course']))

<Graph identifier=Nbd1bf23cf720428bb3564627fb05354c (<class 'rdflib.graph.Graph'>)>

Add an additional triple (which describes the same subject, `node_uri`):

In [8]:
my_kg.add((node_uri, FOAF['name'], Literal('Building Knowledge Graphs')))

<Graph identifier=Nbd1bf23cf720428bb3564627fb05354c (<class 'rdflib.graph.Graph'>)>

And now let's dump our graph triples into some `ttl` file:

In [10]:
my_kg.serialize('sample_graph.ttl', format="turtle")

<Graph identifier=Nbd1bf23cf720428bb3564627fb05354c (<class 'rdflib.graph.Graph'>)>

In [11]:
!head sample_graph.ttl

'head' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
